## 1. Instalación

Asegúrate de tener un archivo .env en el que puedas guardar tus credenciales de la API de Google Natural Language, puedes ver un tutorial aquí donde explican un poco sobre su instalación:

https://www.youtube.com/watch?v=iqRgOdJZtiY

In [2]:
pip install google-cloud-language

  Obtaining dependency information for google-cloud-language from https://files.pythonhosted.org/packages/6b/ba/7700f9c2c373a2a1d633c239deb732fefb6bab2ec72d4da36ecfbd27e691/google_cloud_language-2.17.2-py3-none-any.whl.metadata
  Obtaining dependency information for google-api-core[grpc]!=2.0.*,!=2.1.*,!=2.10.*,!=2.2.*,!=2.3.*,!=2.4.*,!=2.5.*,!=2.6.*,!=2.7.*,!=2.8.*,!=2.9.*,<3.0.0,>=1.34.1 from https://files.pythonhosted.org/packages/14/4b/ead00905132820b623732b175d66354e9d3e69fcf2a5dcdab780664e7896/google_api_core-2.25.1-py3-none-any.whl.metadata
  Obtaining dependency information for proto-plus<2.0.0,>=1.22.3 from https://files.pythonhosted.org/packages/4e/6d/280c4c2ce28b1593a19ad5239c8b826871fc6ec275c21afc8e1820108039/proto_plus-1.26.1-py3-none-any.whl.metadata
  Obtaining dependency information for protobuf!=4.21.0,!=4.21.1,!=4.21.2,!=4.21.3,!=4.21.4,!=4.21.5,<7.0.0,>=3.20.2 from https://files.pythonhosted.org/packages/6a/c9/b9689a2a250264a84e66c46d8862ba788ee7a641cdca39bccf64f5928

In [28]:
pip install --upgrade google-cloud-translate

  Obtaining dependency information for google-cloud-translate from https://files.pythonhosted.org/packages/dc/cc/7d867218b8389ff150847931a9b2415a25e642b8072c3fe57b310f1400d4/google_cloud_translate-3.21.1-py3-none-any.whl.metadata
  Obtaining dependency information for google-cloud-core<3.0.0,>=1.4.4 from https://files.pythonhosted.org/packages/40/86/bda7241a8da2d28a754aad2ba0f6776e35b67e37c36ae0c45d49370f1014/google_cloud_core-2.4.3-py2.py3-none-any.whl.metadata
  Obtaining dependency information for grpc-google-iam-v1<1.0.0,>=0.14.0 from https://files.pythonhosted.org/packages/66/6f/dd9b178aee7835b96c2e63715aba6516a9d50f6bebbd1cc1d32c82a2a6c3/grpc_google_iam_v1-0.14.2-py3-none-any.whl.metadata
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 204.1/204.1 kB 1.3 MB/s eta 0:00:00a 0:00:01
Note: you may need to restart the kernel to use updated packages.


In [3]:
from google.cloud import language_v1 as language

In [5]:
from dotenv import load_dotenv
import os

In [7]:
load_dotenv()  # Carga las variables del archivo .env
credentials_path = os.getenv('GOOGLE_APPLICATION_CREDENTIALS')

os.environ['GOOGLE_APPLICATION_CREDENTIALS'] = credentials_path

In [8]:
client=language.LanguageServiceClient()

## 2. Probemos el Análisis de Sentimiento

In [16]:
import pandas as pd 
def analizar_sentimiento_texto(texto: str):
    cliente = language.LanguageServiceClient()
    documento = language.Document(
        content=texto,
        type_=language.Document.Type.PLAIN_TEXT,
    )
    respuesta = cliente.analyze_sentiment(document=documento)
    return respuesta

def obtener_dataframes_sentimiento(respuesta: language.AnalyzeSentimentResponse):
    # DataFrame para oraciones
    oraciones = [{
        "oracion": s.text.content,
        "score": s.sentiment.score,
        "magnitud": s.sentiment.magnitude
    } for s in respuesta.sentences]
    df_oraciones = pd.DataFrame(oraciones)

    # DataFrame para documento completo
    sentimiento_doc = respuesta.document_sentiment
    df_documento = pd.DataFrame([{
        "puntaje": sentimiento_doc.score,
        "magnitud": sentimiento_doc.magnitude,
        "idioma": respuesta.language
    }])

    return df_oraciones, df_documento

In [19]:
texto = (
    "Python es un lenguaje muy legible, lo que facilita entender y mantener el código. "
    "Probablemente blik-182 es una de las mejores bandas de la historia. "
    "Odio trabajar con APIS porque no las entiendo y me causa mucha frustración."
)

respuesta_sentimiento = analizar_sentimiento_texto(texto)
df_oraciones, df_documento = obtener_dataframes_sentimiento(respuesta_sentimiento)

In [20]:
df_oraciones

,oracion,score,magnitud
0,"Python es un lenguaje muy legible, lo que faci...",0.8,0.8
1,Probablemente blik-182 es una de las mejores b...,0.9,0.9
2,Odio trabajar con APIS porque no las entiendo ...,-0.5,0.5



- El Score del sentimiento oscila entre -1,0 (negativo) y 1,0 (positivo) y corresponde al sentimiento general a partir de la información proporcionada.
- La magnitud del sentimiento oscila entre 0,0 y $+\infty$ e indica la intensidad general del sentimiento a partir de la información proporcionada. Cuanta más información se proporcione, mayor será la magnitud.

## 3. Análisis de entidades:
En este apartado se tienen un par de funciones y cómo la API nos ayuda a analizar nuestro texto y descomponerlo en sus principales entidades y el tipo que respecta a cada una.

In [21]:
def analizar_entidades_texto(texto: str) -> language.AnalyzeEntitiesResponse:
    cliente = language.LanguageServiceClient()
    documento = language.Document(
        content=texto,
        type_=language.Document.Type.PLAIN_TEXT,
    )
    return cliente.analyze_entities(document=documento)

def mostrar_entidades_texto_completo(respuesta: language.AnalyzeEntitiesResponse) -> pd.DataFrame:
    datos = []
    for entidad in respuesta.entities:
        # Extraer todas las menciones (texto y tipo)
        menciones = []
        for m in entidad.mentions:
            tipo_mencion = language.EntityMention.Type(m.type_).name
            menciones.append(f"{m.text.content} ({tipo_mencion})")
        menciones_str = "; ".join(menciones)

        datos.append({
            "nombre": entidad.name,
            "tipo": language.Entity.Type(entidad.type_).name,
            "relevancia": entidad.salience,
            "mid": entidad.metadata.get("mid", ""),
            "url_wikipedia": entidad.metadata.get("wikipedia_url", ""),
            "url_google": entidad.metadata.get("google_url", ""),
            "menciones": menciones_str,
        })

    df = pd.DataFrame(datos)
    if not df.empty:
        df["relevancia"] = df["relevancia"].apply(lambda x: f"{x:.1%}")
    return df


In [22]:
texto = "The Story So Far es una banda de pop-punk liderada por Parker Cannon. Han participado en festivales importantes como el Vans Warped Tour."

respuesta = analizar_entidades_texto(texto)
df_entidades = mostrar_entidades_texto_completo(respuesta)
df_entidades

,nombre,tipo,relevancia,mid,url_wikipedia,url_google,menciones
0,banda,ORGANIZATION,32.7%,,,,banda (COMMON)
1,The Story So Far,WORK_OF_ART,20.1%,/m/01v5yd1,https://en.wikipedia.org/wiki/The_Story_So_Far...,,The Story So Far (PROPER)
2,Parker Cannon,PERSON,19.7%,/m/01081vns,,,Parker Cannon (PROPER)
3,pop-punk,OTHER,14.0%,,,,pop-punk (COMMON)
4,festivales,EVENT,8.7%,,,,festivales (COMMON)
5,Vans Warped Tour,EVENT,4.8%,/m/017bc3,https://en.wikipedia.org/wiki/Warped_Tour,,Vans Warped Tour (PROPER)
6,una,NUMBER,0.0%,,,,una (TYPE_UNKNOWN)


## 4. Clasificación de Contenido


In [41]:
from google.cloud import language_v1 as language
from google.cloud import translate_v2 as translate
import pandas as pd

def traducir_texto(texto: str, idioma_destino: str = "en") -> str:
    cliente_traductor = translate.Client()
    resultado = cliente_traductor.translate(texto, target_language=idioma_destino)
    return resultado["translatedText"]

def clasificar_texto(texto: str) -> language.ClassifyTextResponse:
    cliente = language.LanguageServiceClient()
    documento = language.Document(
        content=texto,
        type_=language.Document.Type.PLAIN_TEXT,
        language="en"  # Clasificación solo soporta inglés
    )
    return cliente.classify_text(document=documento)

def mostrar_clasificacion_texto(respuesta: language.ClassifyTextResponse) -> pd.DataFrame:
    datos = [{
        "categoría": categoria.name,
        "confianza": categoria.confidence
    } for categoria in respuesta.categories]

    df = pd.DataFrame(datos)
    if not df.empty:
        df["confianza"] = df["confianza"].apply(lambda x: f"{x:.0%}")
    return df

In [45]:
texto = "Oppo Reno ha lanzado su nueva línea de telefónos que te volará la cabeza con sus nuevas modalidades de inteligencia artificial."

respuesta = clasificar_texto(texto)
df_clasificacion = mostrar_clasificacion_texto(respuesta)
df_clasificacion

,categoría,confianza
0,/Computers & Electronics/Consumer Electronics,95%
1,/Internet & Telecom,51%


## 4. Moderación de Textos 
la moderación de texto detecta una amplia variedad de contenidos dañinos, incluyendo discursos de odio, acoso y hostigamiento sexual. Este análisis se realiza mediante el método moderate_text, que devuelve una respuesta del tipo ModerateTextResponse.

In [46]:
def moderar_texto(texto: str) -> language.ModerateTextResponse:
    cliente = language.LanguageServiceClient()
    documento = language.Document(
        content=texto,
        type_=language.Document.Type.PLAIN_TEXT,
    )
    return cliente.moderate_text(document=documento)

def mostrar_moderacion_texto(respuesta: language.ModerateTextResponse) -> pd.DataFrame:
    categorias = respuesta.moderation_categories

    # Ordenar por confianza descendente
    categorias_ordenadas = sorted(categorias, key=lambda c: c.confidence, reverse=True)

    datos = [{
        "categoría": categoria.name,
        "confianza": categoria.confidence
    } for categoria in categorias_ordenadas]

    df = pd.DataFrame(datos)
    if not df.empty:
        df["confianza"] = df["confianza"].apply(lambda x: f"{x:.0%}")
    return df

In [54]:
texto = ("Hola, los odio a todos.")

respuesta_moderacion = moderar_texto(texto)
df_moderacion = mostrar_moderacion_texto(respuesta_moderacion)
df_moderacion

,categoría,confianza
0,Violent,84%
1,Toxic,72%
2,Illicit Drugs,70%
3,Derogatory,54%
4,Insult,44%
5,Firearms & Weapons,43%
6,Religion & Belief,35%
7,Profanity,29%
8,War & Conflict,21%
9,"Death, Harm & Tragedy",19%
